In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
def withStar(string):
  if string=='':
    return '';
  else:
    return '**'+string[:]+'**'

In [3]:
def grams(string,num)->list[str]:
    string=withStar(string)
    begin=len(string)-num;
    token=[];
    for i in range(begin+1):
        a=[];
        a=string[i:i+3]
        token.append(a);
    return token;

In [4]:
def overlap(string1,string2)->int:
    s1=grams(string1,3);
    s2=grams(string2,3);
    common_elements = list(set(s1).intersection(set(s2)));
    lc=len(common_elements);
    m=min(len(s1),len(s2));
    if m==0: 
        return 0;
    else:
        sim1=lc/m;
        return sim1;

In [5]:
def jaccard(string1,string2):
    s1=grams(string1,3);
    s2=grams(string2,3);
    common_elements = list(set(s1).intersection(set(s2)));
    lc=len(common_elements);
    union_elements=list(set(s1).union(set(s2)));
    lu=len(union_elements);
    if lu==0:
        return 0;
    else:
        sim2 = lc/lu;
        return sim2;

In [6]:
def dice(string1,string2):
    s1=grams(string1,3);
    s2=grams(string2,3);
    common_elements = list(set(s1).intersection(set(s2)));
    lc=len(common_elements);
    if len(s1)+len(s2)==0:
        return 0;
    else:
        sim3=2*lc/(len(s1)+len(s2));
        return sim3;

In [7]:
def accuracy(mark_real,mark_test):
    sum = 0
    for i in range(len(mark_real)):
        if mark_real[i]==mark_test[i]:
            sum=sum+1
        else:
            continue
    return sum/len(mark_test)

In [8]:
buy_train = pd.read_csv("buy_train.tsv", sep="\t", usecols=['idx','text_left','text_right','label'])
amazon_train = pd.read_csv("amazonGoogle_train.tsv", sep="\t", usecols=['idx','text_left','text_right','label'])
dirty_amazon_train = pd.read_csv("dirty_amazon_train.tsv", sep="\t")
acm_train = pd.read_csv("acm_train.tsv", sep="\t")
walmart_train = pd.read_csv("walmart_train.tsv", sep="\t")
scholar_train = pd.read_csv("scholar_train.tsv", sep="\t")

In [10]:
result_train = pd.concat([buy_train,amazon_train,dirty_amazon_train,acm_train,walmart_train,scholar_train], axis=0)
result_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43722 entries, 0 to 17222
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   idx         43722 non-null  int64 
 1   text_left   43722 non-null  object
 2   text_right  43722 non-null  object
 3   label       43722 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


In [11]:
result_train['idx']=range(len(result_train['idx']))

In [12]:
result_train.index=range(len(result_train['idx']))

In [13]:
result_train

,idx,text_left,text_right,label
0,0,lg 24 ' lds4821ww semi integrated built in whi...,lg ldf6920bb fully integrated dishwasher,0
1,1,speck seethru clear hard shell case for macboo...,speck products seethru case for apple 13 ' mac...,0
2,2,denon blu-ray disc dvd/cd player dvd3800bdci 1...,denon dvd-2930ci dvd player dvd2930ci dvd + rw...,0
3,3,panasonic dect 6.0 expandable digital cordless...,panasonic kx-tg1032s dual handset digital cord...,0
4,4,sony silver minidv handycam camcorder dcrhc52 ...,sony minidv head cleaner dvm12cld head cleaner,0
...,...,...,...,...
43717,43717,a simple algorithm for finding frequent elemen...,an algorithm for in-core frequent itemset mini...,0
43718,43718,"relaxed transaction processing m singh , c tom...",persistent linda : linda + transactions + quer...,0
43719,43719,deeds towards a distributed and active real-ti...,distributed multimedia database systems suppor...,0
43720,43720,a transducer-based xml query processor b ludфs...,"a transducer-based xml query processor , p muk...",1


In [27]:
question1=result_train['text_left']
question2=result_train['text_right']
mark=result_train['label']

In [28]:
# #data processing
# question1_token=[]
# question2_token=[]
# for i in range(len(question1)):
#     question1_token.append(grams(question1[i],3))
# for i in range(len(question2)):
#     question2_token.append(grams(question2[i],3))
question1

0        lg 24 ' lds4821ww semi integrated built in whi...
1        speck seethru clear hard shell case for macboo...
2        denon blu-ray disc dvd/cd player dvd3800bdci 1...
3        panasonic dect 6.0 expandable digital cordless...
4        sony silver minidv handycam camcorder dcrhc52 ...
                               ...                        
43717    a simple algorithm for finding frequent elemen...
43718    relaxed transaction processing m singh , c tom...
43719    deeds towards a distributed and active real-ti...
43720    a transducer-based xml query processor b ludфs...
43721    improving database design through the analysis...
Name: text_left, Length: 43722, dtype: object

In [29]:
question2

0                 lg ldf6920bb fully integrated dishwasher
1        speck products seethru case for apple 13 ' mac...
2        denon dvd-2930ci dvd player dvd2930ci dvd + rw...
3        panasonic kx-tg1032s dual handset digital cord...
4           sony minidv head cleaner dvm12cld head cleaner
                               ...                        
43717    an algorithm for in-core frequent itemset mini...
43718    persistent linda : linda + transactions + quer...
43719    distributed multimedia database systems suppor...
43720    a transducer-based xml query processor , p muk...
43721    the design of the triton nested relational dat...
Name: text_right, Length: 43722, dtype: object

In [30]:
#calculate the overlap coefficient
overlap_coefficient=[]
for i in range(len(question1)):
    overlap_coefficient.append(overlap(question1[i],question2[i]))

In [31]:
#get the equal coefficient
equal_overlap=[]
for i in range(len(overlap_coefficient)):
    if mark[i]==1:
        equal_overlap.append(overlap_coefficient[i])
#         print("The two sentence is equal, overlap coefficient is:",overlap_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [32]:
#get the equal coefficient
inequal_overlap=[]
for i in range(len(overlap_coefficient)):
    if mark[i]==0:
        inequal_overlap.append(overlap_coefficient[i])
#         print("The two sentence is not equal, overlap coefficient is:",overlap_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [33]:
thershold=[]
for i in range(1,100):
    thershold.append(0.01*i)

In [34]:
accuracy_overlap_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(overlap_coefficient)):
        if overlap_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    accuracy_overlap_train.append(accuracy(mark,mark_train_pred))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max accuracy is: {}".format(max(accuracy_overlap_train)))
    print("thershold of the max accuracy is: {}".format(accuracy_overlap_train.index(max(accuracy_overlap_train))))#0.68

Thershold is: 0.01
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.02
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.03
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.04
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.05
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.06
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.07
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.08
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.09
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.10
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold 

Thershold is: 0.83
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.84
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.85
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.86
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.87
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.88
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.89
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.90
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.91
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67
Thershold is: 0.92
current max accuracy is: 0.9271762499428206
thershold of the max accuracy is: 67


In [35]:
from sklearn.metrics import f1_score
fscore_overlap_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(overlap_coefficient)):
        if overlap_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    fscore_overlap_train.append(f1_score(mark,mark_train_pred,average='binary'))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max f-score is: {}".format(max(fscore_overlap_train)))
    print("thershold of the max fscore is: {}".format(fscore_overlap_train.index(max(fscore_overlap_train))))#0.67

Thershold is: 0.01
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.02
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.03
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.04
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.05
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.06
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.07
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.08
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.09
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.10
current max f-score is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.11
current max f-score i

Thershold is: 0.91
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.92
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.93
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.94
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.95
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.96
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.97
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.98
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66
Thershold is: 0.99
current max f-score is: 0.749243306169965
thershold of the max fscore is: 66


In [ ]:
# for i in range(len(overlap_coefficient)):
#     if overlap_coefficient[i] in equal_overlap:
#         plt.scatter(i,overlap_coefficient[i],color='r')
#     else:
#         plt.scatter(i,overlap_coefficient[i],color='b')
# ##thershold 

In [ ]:
# x=np.linspace(0,1,len(inequal_overlap))
# plt.plot(x,inequal_overlap)

In [36]:
#calculate the jaccard coefficient
jaccard_coefficient=[]
for i in range(len(question1)):
    jaccard_coefficient.append(jaccard(question1[i],question2[i]))

In [37]:
#get the equal coefficient
equal_jaccard=[]
for i in range(len(jaccard_coefficient)):
    if mark[i]==1:
        equal_jaccard.append(jaccard_coefficient[i])
#         print("The two sentence is equal, overlap coefficient is:",jaccard_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [38]:
#get the inequal coefficient
inequal_jaccard=[]
for i in range(len(jaccard_coefficient)):
    if mark[i]==0:
        inequal_jaccard.append(jaccard_coefficient[i])
#         print("The two sentence is not equal, overlap coefficient is:",jaccard_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [39]:
accuracy_jaccard_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(jaccard_coefficient)):
        if jaccard_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    accuracy_jaccard_train.append(accuracy(mark,mark_train_pred))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max accuracy is: {}".format(max(accuracy_jaccard_train)))
    print("thershold of the max accuracy is: {}".format(accuracy_jaccard_train.index(max(accuracy_jaccard_train))))#0.49

Thershold is: 0.01
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.02
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.03
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.04
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.05
current max accuracy is: 0.14930698504185536
thershold of the max accuracy is: 4
Thershold is: 0.06
current max accuracy is: 0.15129682997118155
thershold of the max accuracy is: 5
Thershold is: 0.07
current max accuracy is: 0.15417867435158503
thershold of the max accuracy is: 6
Thershold is: 0.08
current max accuracy is: 0.15895887653812726
thershold of the max accuracy is: 7
Thershold is: 0.09
current max accuracy is: 0.16682676913224465
thershold of the max accuracy is: 8
Thershold is: 0.10
current max accuracy is: 0.17725630117560953
thershold of the max accuracy is: 9
Ther

Thershold is: 0.83
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.84
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.85
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.86
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.87
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.88
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.89
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.90
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.91
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48
Thershold is: 0.92
current max accuracy is: 0.9303096839119894
thershold of the max accuracy is: 48


In [40]:
from sklearn.metrics import f1_score
fscore_jaccard_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(jaccard_coefficient)):
        if jaccard_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    fscore_jaccard_train.append(f1_score(mark,mark_train_pred,average='binary'))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max fscore is: {}".format(max(fscore_jaccard_train)))
    print("thershold of the max fscore is: {}".format(fscore_jaccard_train.index(max(fscore_jaccard_train))))#0.43

Thershold is: 0.01
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.02
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.03
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.04
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.05
current max fscore is: 0.2592312288388767
thershold of the max fscore is: 4
Thershold is: 0.06
current max fscore is: 0.25959255342498555
thershold of the max fscore is: 5
Thershold is: 0.07
current max fscore is: 0.2601580474142242
thershold of the max fscore is: 6
Thershold is: 0.08
current max fscore is: 0.2612504018000643
thershold of the max fscore is: 7
Thershold is: 0.09
current max fscore is: 0.2629491744901263
thershold of the max fscore is: 8
Thershold is: 0.10
current max fscore is: 0.26512768130745656
thershold of the max fscore is: 9
Thershold is: 0.11
current max fscore is: 0.2687

Thershold is: 0.91
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.92
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.93
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.94
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.95
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.96
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.97
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.98
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42
Thershold is: 0.99
current max fscore is: 0.7479473276529822
thershold of the max fscore is: 42


In [ ]:
# for i in range(len(jaccard_coefficient)):
#     if jaccard_coefficient[i] in inequal_jaccard:
#         plt.scatter(i,jaccard_coefficient[i],color='r')
#     else:
#         plt.scatter(i,jaccard_coefficient[i],color='b')
# # worse than overlap coeffient

In [41]:
#calculate the dice's coefficient
dice_coefficient=[]
for i in range(len(question1)):
    dice_coefficient.append(dice(question1[i],question2[i]))

In [42]:
#get the equal coefficient
equal_dice=[]
for i in range(len(dice_coefficient)):
    if mark[i]==1:
        equal_dice.append(dice_coefficient[i])
#         print("The two sentence is equal, overlap coefficient is:",dice_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [43]:
#get the inequal coefficient
inequal_dice=[]
for i in range(len(dice_coefficient)):
    if mark[i]==0:
        inequal_dice.append(dice_coefficient[i])
#         print("The two sentence is not equal, overlap coefficient is:",dice_coefficient[i])
#         print(question1[i])
#         print(question2[i])
#         print('-'*50)

In [44]:
accuracy_dice_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(dice_coefficient)):
        if dice_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    accuracy_dice_train.append(accuracy(mark,mark_train_pred))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max accuracy is: {}".format(max(accuracy_dice_train)))
    print("thershold of the max accuracy is: {}".format(accuracy_dice_train.index(max(accuracy_dice_train))))#0.59

Thershold is: 0.01
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.02
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.03
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.04
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.05
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.06
current max accuracy is: 0.1488495494259183
thershold of the max accuracy is: 0
Thershold is: 0.07
current max accuracy is: 0.148895292987512
thershold of the max accuracy is: 6
Thershold is: 0.08
current max accuracy is: 0.1492841132610585
thershold of the max accuracy is: 7
Thershold is: 0.09
current max accuracy is: 0.150450574081698
thershold of the max accuracy is: 8
Thershold is: 0.10
current max accuracy is: 0.1519601116142903
thershold of the max accuracy is: 9
Thershold is

Thershold is: 0.84
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.85
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.86
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.87
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.88
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.89
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.90
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.91
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.92
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58
Thershold is: 0.93
current max accuracy is: 0.9298522482960523
thershold of the max accuracy is: 58


In [45]:
fscore_dice_train=[]
for i in range(len(thershold)):
    mark_train_pred=[]
    for j in range(len(dice_coefficient)):
        if dice_coefficient[j] >= thershold[i]:
            mark_train_pred.append(1)
        else: mark_train_pred.append(0)
    fscore_dice_train.append(f1_score(mark,mark_train_pred,average='binary'))
    print("Thershold is: {:.2f}".format(thershold[i]))
    print("current max fscore is: {}".format(max(fscore_dice_train)))
    print("thershold of the max fscore is: {}".format(fscore_dice_train.index(max(fscore_dice_train))))#0.56

Thershold is: 0.01
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.02
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.03
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.04
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.05
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.06
current max fscore is: 0.2591280111487159
thershold of the max fscore is: 0
Thershold is: 0.07
current max fscore is: 0.25913832921876245
thershold of the max fscore is: 6
Thershold is: 0.08
current max fscore is: 0.2592260660014738
thershold of the max fscore is: 7
Thershold is: 0.09
current max fscore is: 0.2594305766010049
thershold of the max fscore is: 8
Thershold is: 0.10
current max fscore is: 0.2597132931358064
thershold of the max fscore is: 9
Thershold is: 0.11
current max fscore is: 0.26018

Thershold is: 0.91
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.92
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.93
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.94
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.95
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.96
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.97
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.98
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55
Thershold is: 0.99
current max fscore is: 0.7502392344497607
thershold of the max fscore is: 55


In [ ]:
# for i in range(len(dice_coefficient)):
#     if dice_coefficient[i] in inequal_dice:
#         plt.scatter(i,dice_coefficient[i],color='r')
#     else:
#         plt.scatter(i,dice_coefficient[i],color='b')

In [ ]:
# # use equal weight in each coefficient
# coefficient=[]
# for i in range(len(mark)):
#     coefficient.append(overlap_coefficient[i]/3+jaccard_coefficient[i]/3+dice_coefficient[i]/3)

In [ ]:
# for i in range(len(coefficient)):
#     if mark[i]==1:
#         plt.scatter(i,coefficient[i],color='r')
#     else:
#         plt.scatter(i,coefficient[i],color='b')
# # same as overlap coefficient

In [46]:
buy_test = pd.read_csv("buy_test.tsv", sep="\t", usecols=['idx','text_left','text_right','label'])
amazon_test = pd.read_csv("amazonGoogle_test.tsv", sep="\t", usecols=['idx','text_left','text_right','label'])
dirty_amazon_test = pd.read_csv("dirty_amazon_test.tsv", sep="\t")
acm_test = pd.read_csv("acm_test.tsv", sep="\t")
walmart_test = pd.read_csv("walmart_test.tsv", sep="\t")
scholar_test = pd.read_csv("scholar_test.tsv", sep="\t")

In [47]:
result_test = pd.concat([buy_test,amazon_test,dirty_amazon_test,acm_test,walmart_test,scholar_test], axis=0)
result_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14582 entries, 0 to 5741
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   idx         14582 non-null  int64 
 1   text_left   14582 non-null  object
 2   text_right  14582 non-null  object
 3   label       14582 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 569.6+ KB


In [48]:
result_test.index=range(len(result_test['idx']))
result_test['idx']=range(len(result_test['idx']))
result_test

,idx,text_left,text_right,label
0,0,sony pink cyber-shot 7.2 megapixel digital cam...,olympus fe-360 digital camera pink 226540 8 me...,0
1,1,lg 2.0 cu . ft. over-the-range white microwave...,maytag 2.0 cu . ft. over-the-range microwave oven,0
2,2,panasonic black 8.5 ' portable dvd player dvdl...,toshiba sd-p71s portable dvd player toshiba sd...,0
3,3,sony bravia theater black micro system davis50...,sony bravia dav-is50 / b home theater system d...,1
4,4,panasonic dect 6.0 expandable digital cordless...,panasonic kx-tg9342t cordless phone 1 x phone ...,0
...,...,...,...,...
14577,14577,incremental clustering for mining in a data wa...,incremental organization for data recording an...,0
14578,14578,mining generalized association rules vldb r sr...,sql-like operator for mining association rules...,0
14579,14579,the design and implementation of a sequence da...,on the design and implementation of informatio...,0
14580,14580,querying multiple features of groups in relati...,the theory of joins in relational databases av...,0


In [49]:
question1_test=result_test['text_left']
question2_test=result_test['text_right']
mark_test=result_test['label']

In [50]:
test_overlap=[]
test_jaccard=[]
test_dice=[]
for i in range(len(question1_test)):
    test_overlap.append(overlap(question1_test[i],question2_test[i]))
    test_jaccard.append(jaccard(question1_test[i],question2_test[i]))
    test_dice.append(dice(question1_test[i],question2_test[i]))

In [ ]:
# #calculate the overlap coefficient
# coefficient_test=[]
# for i in range(len(question1_test)):
#     coefficient_test.append(overlap(question1_test[i],question2_test[i])/3+jaccard(question1_test[i],question2_test[i])/3+dice(question1_test[i],question2_test[i])/3)

In [ ]:
#     coefficient_test.append(overlap(question1_test[i],question2_test[i])/3+jaccard(question1_test[i],question2_test[i])/3+dice(question1_test[i],question2_test[i])/3)

In [ ]:
# for i in range(len(coefficient_test)):
#     if mark_test[i]==1:
#         plt.scatter(i,coefficient_test[i],color='r')
#     else:
#         plt.scatter(i,coefficient_test[i],color='b')

In [51]:
def calculate_is_duplicate(coefficient,thershold):
    mark=[]
    for i in range(len(coefficient)):
        if coefficient[i] >= thershold:
            mark.append(1)
        else:
            mark.append(0)
    return mark

In [52]:
mark_overlap_acc_test=calculate_is_duplicate(test_overlap,0.68)
print(accuracy(mark_test,mark_overlap_acc_test))
print('-'*50)
mark_overlap_f_test=calculate_is_duplicate(test_overlap,0.67)
print("fscore:")
print(f1_score(mark_test,mark_overlap_f_test,average='binary'))

0.9286791935262653
--------------------------------------------------
fscore:
0.7563143124415341


In [53]:
mark_jaccard_acc_test=calculate_is_duplicate(test_jaccard,0.49)
print(accuracy(mark_test,mark_jaccard_acc_test))
print('-'*50)
mark_jaccard_f_test=calculate_is_duplicate(test_jaccard,0.43)
print("fscore:")
print(f1_score(mark_test,mark_jaccard_f_test,average='binary'))

0.9299821697983816
--------------------------------------------------
fscore:
0.7494766224703419


In [54]:
mark_dice_acc_test=calculate_is_duplicate(test_dice,0.59)
print(accuracy(mark_test,mark_dice_acc_test))
print('-'*50)
mark_dice_f_test=calculate_is_duplicate(test_dice,0.56)
print("fscore:")
print(f1_score(mark_test,mark_dice_f_test,average='binary'))

0.9308051021807708
--------------------------------------------------
fscore:
0.7542637521018496


In [ ]:
# def acc(mark_real,mark_test):
#     sum = 0
#     for i in range(len(mark_real)):
#         if mark_real[i]==mark_test[i]:
#             sum=sum+1
#         else:
#             continue
#     return sum/len(mark_test)

In [ ]:
# acc(mark_test,mark_calculate)

In [ ]:
# use a classifier to predict the result
X_train=[]
X_test=[]
for i in range(len(question1)):
    co=(overlap_coefficient[i], jaccard_coefficient[i], dice_coefficient[i])
    X_train.append(co)
y_train=mark
for i in range(len(question1_test)):
    co=(test_overlap[i], test_jaccard[i], test_dice[i])
    X_test.append(co)
y_test=mark_test

In [ ]:
#use KNeighbors Classifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=1)
model=knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print("Test set predictions:\n{}".format(y_pred))
print("Test set score: {:.2f}".format(np.mean(y_pred == y_test)))
print("Test set score: {:.2f}".format(knn.score(X_test, y_test)))